<a href="https://colab.research.google.com/github/farhanreynaldo/rethinking/blob/master/rethinking_chapter_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages used in this notebook
import os
import json
import shutil
import urllib.request
import pandas as pd

In [2]:
# Install package CmdStanPy
!pip install --upgrade cmdstanpy

     |████████████████████████████████| 40kB 2.8MB/s 
ERROR: fbprophet 0.6 has requirement cmdstanpy==0.4, but you'll have cmdstanpy 0.9.62 which is incompatible.
  Found existing installation: cmdstanpy 0.4.0
    Uninstalling cmdstanpy-0.4.0:
      Successfully uninstalled cmdstanpy-0.4.0


In [3]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
# https://mc-stan.org/users/documentation/case-studies/jupyter_colab_notebooks_2020.html
# https://github.com/stan-dev/example-models/blob/master/knitr/cloud-compute-2020/CmdStanPy_Example_Notebook.ipynb

tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'

urllib.request.urlretrieve(tgz_url, tgz_file)
shutil.unpack_archive(tgz_file)

# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'

In [4]:
# Check CmdStan path
from cmdstanpy import cmdstan_path
cmdstan_path()

'./cmdstan-2.23.0'

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

In [39]:
howell1_url = 'https://raw.githubusercontent.com/pymc-devs/resources/master/Rethinking_2/Data/Howell1.csv'
df = pd.read_csv(howell1_url, sep=';', header=0).loc[lambda df: df['age'] >= 18]

In [40]:
data = df.filter(['height']).to_dict(orient='list')
data['n'] = df.shape[0]

In [41]:
%%file m41.stan

data {
  int<lower=1> n;
  vector[n] height;
}
parameters {
  real mu;
  real<lower=0,upper=50> sigma;
}
model {
  height ~ normal(mu, sigma);
  sigma ~ uniform(0, 50);
  mu ~ normal(178, 20);
}

Overwriting stan_program.stan


In [44]:
m41 = CmdStanModel(stan_file='m41.stan')
fit = m41.sample(data=data)
fit.summary()

INFO:cmdstanpy:compiling stan program, exe file: /content/stan_program
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /content/stan_program
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-895.75900,0.024281,1.002590,-897.82200,-895.43400,-894.7940,1705.02,10440.4,1.001940
mu,154.60600,0.006984,0.414337,153.91500,154.60600,155.2920,3520.00,21554.2,1.000320
sigma,7.76938,0.004802,0.297174,7.28377,7.76599,8.2795,3830.36,23454.7,0.999638
